In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

In [ ]:
%%cuda --name SDL.cu

#include "/content/drive/MyDrive/graph/coloring.h"
#include "/content/drive/MyDrive/graph/graph_d.h"

#define THREADxBLOCK 128

using namespace std;

__device__ bool result;

__global__ void findIS (Coloring* col, GraphStruct *str, uint* weigths, bool* candidateNodes) {
	uint idx = threadIdx.x + blockDim.x * blockIdx.x;

	if (idx >= str->nodeSize)
		return;

	if (col->coloring[idx])
		return;

	uint offset = str->cumDegs[idx];
	uint deg = str->cumDegs[idx + 1] - str->cumDegs[idx];

	bool candidate = true;
  for (uint j = 0; j < deg; j++) {
		uint neighID = str->neighs[offset + j];
    uint neighDeg = str->cumDegs[neighID + 1] - str->cumDegs[neighID];
		if (col->coloring[neighID] == 0 &&
				((deg < neighDeg) ||
				((deg == neighDeg) && (weigths[idx] < weigths[neighID])))) {
			candidate = false;
		}
	}


 if(candidate){
		candidateNodes[idx] = true;
  }

}

__global__ void colorer (Coloring* col, GraphStruct *str, bool* candidateNodes) {
	uint idx = threadIdx.x + blockDim.x * blockIdx.x;

	if (idx >= str->nodeSize)
		return;

	if (col->coloring[idx])
		return;

	uint offset = str->cumDegs[idx];
	uint deg = str->cumDegs[idx + 1] - str->cumDegs[idx];

  if(candidateNodes[idx]){
      bool* forbidden;
			uint n = str->nodeSize;
			cudaMalloc((void**) &forbidden, n * sizeof(bool));
			memset(forbidden, false, n);

			for (uint j = 0; j < deg; j++) {
					uint neighID = str->neighs[offset + j];
					forbidden[col->coloring[neighID]] = true;
			}

			for(uint i = 0; i <= n; i++){
				if(forbidden[i] == false){
					col->coloring[idx] = i;
					free(forbidden);
					return;
				}
			}
		free(forbidden);
	}

	else
	 		col->uncoloredNodes = true;

}

Coloring* graphColoring(GraphStruct *str) {
  Coloring* col;
	CHECK(cudaMallocManaged(&col, sizeof(Coloring)));
	uint n = str->nodeSize;
	col->uncoloredNodes = true;

  CHECK(cudaMallocManaged( &(col->coloring), n * sizeof(uint)));
	memset(col->coloring,0,n);

	uint* weigths;
  curandState_t* states;
	cudaMalloc((void**) &states, n * sizeof(curandState_t));
	cudaMalloc((void**) &weigths, n * sizeof(uint));
	dim3 threads ( THREADxBLOCK);
	dim3 blocks ((str->nodeSize + threads.x - 1) / threads.x, 1, 1 );
	uint seed = 0;

	init <<< blocks, threads >>> (seed, states, weigths, n);

	bool* candidateNodes;
	cudaMalloc((void**) &candidateNodes, n * sizeof(bool));
	cudaMemset(candidateNodes, false, n);

  col->numOfColors = 0;
	while (col->uncoloredNodes) {
		col->uncoloredNodes = false;
    col->numOfColors++;
		findIS <<< blocks, threads >>> (col, str, weigths, candidateNodes);
		colorer <<< blocks, threads >>> (col, str, candidateNodes);
		cudaDeviceSynchronize();
	}

	cudaFree(states);
	return col;
}

__global__ void checkUnweightedNodes(GraphStruct *str, bool* unweightedNodes, bool * result) {
		uint idx = threadIdx.x + blockDim.x * blockIdx.x;
		if (idx >= str->nodeSize)
				return;

    if(unweightedNodes[idx] == false)
      result = true;
}

__global__ void init (uint seed, curandState_t* states, uint* weigths, uint n) {
    uint degree = 1;
    uint weigth = 1;
    bool * unweightedNodes;
    cudaMalloc((void**) &unweightedNodes, n * sizeof(bool));
    memset(unweightedNodes,false,n);

		uint idx = threadIdx.x + blockDim.x * blockIdx.x;

		if(idx > n){
			return;
		}

    bool result_h = true;
	  cudaMemcpyToSymbol(result, &result_h, sizeof(bool));

    while(h_result){
        h_result = false;

        for


        checkUnweightedNodes <<< blocks, threads >>> (str, unweightedNodes, h_result);
    }





		weigths[idx] = idx;
}

In [ ]:
#include <stdio.h>
#include <cuda.h>
#include <iostream>
#include <vector>
#include <set>

using namespace std;

void createGraph(int &n, int &m, int &maxDegree, int &minDegree, int** h_degree, int** h_adj, int** h_adj_p)
{
    int i, ch;
    cout << "Enter number of vertices\n";
    cin >> n;
    cout << "Enter number of edges\n";
    cin >> m;
    int k = ((n * (n - 1)) / 2);
    if(k < m){
        cout << "Too many edges...making it complete graph." << endl;
        m = k;
    }
    cout << "Enter 0 for random and 1 for manual graph" << endl;
    cin >> ch;
    vector<set<int>> adj(n);

    // Popolo le liste di adiacenza
    if(ch == 1){
        for(i = 0;i < m;i++)
        {
            // cout << "Enter edge (0 is the first vertex)\n";
            int u,v;
            cin >> u >> v;
            if(adj[u].find(v) != adj[u].end()){
                // cout << "Edge already present" << endl;
                i--;
                continue;
            }
            adj[u].insert(v);
            adj[v].insert(u);
        }
    }
    else if(ch == 0){
        i = 0;
        while(i < m){
            int u = rand() % n;
            int v = rand() % n;
            if(adj[u].find(v) != adj[u].end())
                continue;
            if(u == v)
                continue;
            adj[u].insert(v);
            adj[v].insert(u);
            i++;
        }
    }
    *h_adj_p = new int[n + 1];      // Lista dei vertici: l'i-esimo elemento del vettore fornisce l'indice che rappresenta l'offset nella lista di adiacenza sotto
    *h_adj = new int[(2 * m) + 1];  // Lista contenente la concatenzazione di tutte le liste di adiacenza di ciascun vertice
    *h_degree = new int[n];         // Lista con i gradi di ciascun vertice

    int point = 0;
    for(int i = 0;i < n; i++){
        (*h_adj_p)[i] = point;      // Salvo l'offset
        for(auto j : adj[i])
            (*h_adj)[point++] = j;  // Popolo la lista di adiacenza lato host e contemporaneamente muovo l'offset
    }

    // Popolo lista dei gradi, salvando anche massimo e minimo
    (*h_adj_p)[n] = point;
    int mx = INT_MIN, mn = INT_MAX;
    for(int i = 0;i < n; i++)
    {
        (*h_degree)[i] = (int)adj[i].size();
        mx = max(mx, (int)adj[i].size());
        mn = min(mn, (int)adj[i].size());
    }
    minDegree = mn;
    maxDegree = mx;
    // cout << "Max degree is : " << mx << " and Min degree is : " << mn << endl;
}

// Assegna i pesi in base al grado di ciascun nodo
__global__ void assignWeightKernel(int n, int k, int weight, int* d_adj, int* d_adj_p, bool* d_visited, int* d_weights)
{
    int index = (blockDim.x * blockIdx.x) + threadIdx.x;

    if( index >= n || d_visited[index] )
        return;

    int deg = 0;
    int start = d_adj_p[index];
    int end = d_adj_p[index + 1];
    for(int i = start; i < end; i++)
    {
        int neighbor = d_adj[i];
        if( !d_visited[neighbor] )
            deg++;
    }
    __syncthreads();

    if(deg <= k)
    {
        d_visited[index] = true;
        d_weights[index] = weight;
    }
}

void randomWeightAssign(int* h_rweights, int n){                                             //Assign random weights to the vertices
    int i, j = 1;
    vector<int> arr(n);
    for(i = 0; i < n; i++)
        arr[i] = i;
    i = 0;
    while(i < n){
        int x = rand() % arr.size();
        h_rweights[arr[x]] = j;
        j++;
        arr.erase(arr.begin() + x);
        i++;
    }
    return;
}

// Assegna i pesi in maniera parallela
void assignWeight(int n, int* d_adj, int* d_adj_p, int* h_weights, bool* h_visited)
{
    // Inizializzazione lato device + copia (non basterebbe una cudaMemSet?)
    int* d_weights=NULL;
    bool* d_visited=NULL;

    cudaMalloc((void**)&d_weights, sizeof(int) * n);
    cudaMalloc((void**)&d_visited, sizeof(int) * n);

    cudaMemcpy(d_weights, h_weights, sizeof(int) * n, cudaMemcpyHostToDevice);
    cudaMemcpy(d_visited, h_visited, sizeof(bool) * n, cudaMemcpyHostToDevice);

    // Assegna i pesi in base al grado del nodo invocando una griglia al più con 512 blocchi ciascuno con 512 thread (?). Se un'unica invocazione di kernel non è sufficiente, il processo viene ripetuto, da qui il ciclo,
    int k = 1;
    int weight = 1;
    int cnt = 1;
    int mx = min(n, 512);

    while(cnt != 0)
    {
        assignWeightKernel<<<mx,mx>>>(n, k, weight, d_adj, d_adj_p, d_visited, d_weights);
        cudaDeviceSynchronize();
        cudaMemcpy(h_visited, d_visited, sizeof(bool) * n, cudaMemcpyDeviceToHost);

        cnt = 0;
        for(int i = 0;i < n; i++)
        {
            if(!h_visited[i])
                cnt++;
        }
        k++;
        weight++;
    }
    cudaMemcpy(h_weights, d_weights, sizeof(int) * n, cudaMemcpyDeviceToHost);
}

__global__ void initializeRemCol(bool* d_rem, int* d_colours, int n){               //Set uncolored vertex set to true and initial colour to invalid(0)
    int ind = (blockDim.x * blockIdx.x) + threadIdx.x;
    if(ind >= n)
        return;
    d_rem[ind] = true;
    d_colours[ind] = 0;
}

__global__ void colorSet(int* d_adj, int* d_adj_p, int* d_weights, bool* d_rem, int* d_colours, int n, int maxDegree){//pass h_deg here
    int index = (blockDim.x * blockIdx.x) + threadIdx.x;
    if(index >= n)
        return;

    if(!d_rem[index])                                                   //If current vertex is already colored, return
        return;

    int i, j, maxColours = maxDegree + 1;

    for(i = d_adj_p[index]; i < d_adj_p[index + 1]; i++){               //Check if any uncolored neighbour has higher weight or if equal, higher index
        j = d_adj[i];
        if(d_rem[j] && ((d_weights[j] > d_weights[index]) || (d_weights[j] == d_weights[index] && j>index)) )
            return;
    }

    d_rem[index] = false;                                               //Current vertex should be coloured so remove it from remaining set

    bool* forbidden = new bool[maxColours + 1];
    for(i = 0; i < maxColours + 1; i++)
        forbidden[i] = false;

    for(i = d_adj_p[index]; i < d_adj_p[index + 1]; i++){               //Find out neighbours colors
        j = d_adj[i];
        forbidden[d_colours[j]] = true;
    }
    for(i = 1; i <= maxColours; i++){                                   //Assign color which is not there in neighbours' colors
        if(!forbidden[i]){
            d_colours[index] = i;
            delete [] forbidden;
            return;
        }
    }
    delete [] forbidden;
}

int* colourGraph(int n, int m, int maxDegree, int* d_adj, int* d_adj_p, int* d_weights){
    int i, rem_size = n, mx = min(n, 512);                                               //Initialize all variables
    bool* h_rem = new bool[n];
    bool* h_inc = new bool[n];
    int* h_colours = new int[n];
    bool* d_rem;
    int *d_colours;

    cudaMalloc((void**)&d_colours, sizeof(int) * n);                                        //Allocate space on GPU
    cudaMalloc((void**)&d_rem, sizeof(bool) * n);

    initializeRemCol<<<mx, mx>>>(d_rem, d_colours, n);

    while(rem_size > 0){
        // cout << rem_size << endl;
        colorSet<<<mx, mx>>>(d_adj, d_adj_p, d_weights, d_rem, d_colours, n, maxDegree);    //Launch kernel
        cudaMemcpy(h_rem, d_rem, sizeof(bool) * n, cudaMemcpyDeviceToHost);                 //Copy back the updated uncolored set
        int k = 0;
        for(i = 0; i < n; i++){
            if(h_rem[i])
                k++;
        }
        rem_size = k;
    }

    cudaMemcpy(h_colours, d_colours, sizeof(int) * n, cudaMemcpyDeviceToHost);

    cudaFree(d_colours);                                                                    //Free memory
    cudaFree(d_rem);

    return h_colours;
}

int main()
{
    // Inizializzo grafo
    int n=0, m=0, maxDegree=0, minDegree=0;

    // Liste di adiacenza, di offset e gradi lato host e device
    int* h_adj = NULL, *h_adj_p = NULL;
    int* d_adj, *d_adj_p;
    int* h_degree = NULL;

    // Creazione del grafo
    createGraph(n, m, maxDegree, minDegree, &h_degree, &h_adj, &h_adj_p);

    // Allocazione memoria lato device + copia da host a device
    cudaMalloc((void**)&d_adj, sizeof(int) * ((2 * m) + 1));
    cudaMalloc((void**)&d_adj_p, sizeof(int) * (n + 1));

    cudaMemcpy(d_adj, h_adj, sizeof(int) * ((2 * m) + 1), cudaMemcpyHostToDevice);
    cudaMemcpy(d_adj_p, h_adj_p, sizeof(int) * (n + 1), cudaMemcpyHostToDevice);

    // Assegnamento dei pesi in maniera parallela in base al grado di ogni nodo
    int* h_weights = new int[n];
    bool* h_visited = new bool[n];

    memset(h_weights, -1, n*sizeof(int));
    memset(h_visited, false, n*sizeof(bool));

    assignWeight(n, d_adj, d_adj_p, h_weights, h_visited);

    int max_wt=INT_MIN;
    for(int i=0;i<n;i++)
    {
        // cout << "Weight of " << i << " is " << h_weights[i] << endl;
        max_wt = max(max_wt, h_weights[i]);
    }

    cout << max_wt << endl;


    int* h_colors = new int[n];
    memset(h_colors,0,sizeof(int)*n);

    int* d_colors, *d_weights;

    cudaMalloc((void**)&d_weights, sizeof(int) * n);
    cudaMalloc((void**)&d_colors, sizeof(int) * n);

    cudaMemcpy(d_weights, h_weights, sizeof(int) * n, cudaMemcpyHostToDevice);
    cudaMemcpy(d_colors, h_colors, sizeof(int) * n, cudaMemcpyHostToDevice);

    // int* h_rweights = new int[n];
    // int* d_rweights;
    // cudaMalloc((void**)&d_rweights, sizeof(int) * n);
    // randomWeightAssign(h_rweights, n);
    // cudaMemcpy(d_rweights, h_rweights, sizeof(int) * n, cudaMemcpyHostToDevice);

    // for(int i=0;i<n;i++)
    //     cout << i << " " << h_rweights[i] << endl;

    h_colors = colourGraph(n, m, maxDegree, d_adj, d_adj_p, d_weights);

    // for(int i=0;i<n;i++)
        // cout << "Color of vertex " << i << " is " << h_colors[i] << endl;

    cudaFree(d_adj);
    cudaFree(d_adj_p);
    cudaFree(d_colors);
    cudaFree(d_weights);
    // cudaFree(d_rweights);
}

In [ ]:
%%cuda --name test_SDL.cu

#include "/content/drive/MyDrive/graph/coloring.h"


int main(void) {
	unsigned int n = 1000;		 // number of nodes for random graphs
	float prob = 0.5;				    // density (percentage) for random graphs
	std::default_random_engine eng{0};  // fixed seed

	srand(time(0));
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

	// new graph with n nodes
	Graph graph(n,1);

	// generate a random graph
	graph.randGraph(prob,eng);

	// get the graph struct
	GraphStruct *str = graph.getStruct();

  cudaEventRecord(start);

	Coloring* col = graphColoring(str);
	cudaDeviceSynchronize();

	cudaEventRecord(stop);
  cudaEventSynchronize(stop);

	//Stampo in millisecondi quanto tempo ci ha messo a colorare il grafo.
  float milliseconds = 0;
  cudaEventElapsedTime(&milliseconds, start, stop);
  printf("%f ms\n", milliseconds);

	//printColoring(col, str, 1);
 printf("Coloratura trovata: ");
	for(int i = 0; i < str->nodeSize; i++){
			printf("%d ", col->coloring[i]);
	}

	return EXIT_SUCCESS;
}